<center> <img src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" align="center" width="650" height="400"/> </center>

<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>

<center><font color=#555555><font size=6> I.F. Gerardo González Flores</font> <br> <br>
    
<center><font color=#555555><font size=6> I.F. Tania Bell Nicholls</font> <br> <br>

<center> <font color= #555555> <font size = 4> April 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/Tania-Bell/MyST_Lab4'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Lab 4 </font> <br> <br> <font color= #555555> <font size = 5>Market's Microstructure </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

<font color= #555555> <font size = 4> Abstract </font>

<font color= #6B6B6B> <font size = 3>  Throughout this laboratory, we will use the information from the Cryptocurrency's Book of Orders from various international exchanges, with the primary objective of visualizing and modeling the market's microstructure through it's characteristics and behaviors like Matchine Engine, Levels, Ask-Bid Volumes, etc.    </font>

<br>

# <font color= #6B6B6B> 0. Install/Load Packages and Depedencies </font>

<hr style="border:0.02in solid gray"> </hr>

## <font color= #6B6B6B> 0.2 Python Packages </font> <font color= #555555>

In order to run this notebook, it is necessary to have installed and/or have the _requirements.txt_ file with the following: 

- pandas
- numpy
- jupyter
- ccxt
- ast
- time
- asyncio

## <font color= #6B6B6B> 0.3 Files Dependencies </font> <font color= #555555>

The following are the file dependencies that are needed to run this notebook:

Because of the incompatability of the jupyter versions the book of orders were saved as csv.

- binance_BTCUSDT.csv (referred to as Exchange 1)
- binance_ETHUSDT.csv (Exchange 2)
- bitfinex_BTCUSDT.csv (Exchange 3)
- bitfinex_ETHUSDT.csv (Exchange 4)
- huobi_BTCUSDT.csv (Exchange 5)
- huobi_ETHUSDT.csv (Exchange 6)

## <font color= #6E6E6E> 0.4 Install Packages</font>

In [51]:
%%capture

# Install all the pip packages in the requirements.txt
import sys
!{sys.executable} -m pip install -r requirements.txt

In [52]:
# importing all libraries
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
import numpy as np
import pandas as pd
import ccxt
import functions as fn
import data as dt
import visualizations

In [53]:
pip install ccxt

# <font color= #6B6B6B> <font size = 5> 1. Introduction </font>

This laboratory focuses on using the information in the cryptocurrencies' Book of Orders to visualize and model the market's microstructure. In order to learn about a market's microstructure it is important to have all the information available, for that reason the market chosen for this project was the cryptocurrecy.

This project starts with obtaining the most accurate and current information of three different exchange markets and three different trading pairs using the ccxt library. The exchange markets chosen were Binance and Bitfinex. The trading pairs chosen were BTC/USDT, ETH/USDT and ADA/USDT. Then the information was organized and summarized taking into account the levels and total bid size and ask size.

For the visualization, graphs of the mid price were generated with the plotly library in order to compare the different time series.

To complete the objetives, the last part of the lab consisted of modeling the microstructure of the market. This was achieved through estimating the Effective Spread based on the Roll model (1984).

# <font color= #6B6B6B> <font size = 5> 2. Objectives </font>

- Obtain, clean and analyze the information obtained from the Book of Orders
- Visualize the mid price time series
- Model the market's microstructure using the effective spread

# <font color= #6B6B6B> <font size = 5> 3. Data Description </font>

<hr style="border:0.02in solid gray"> </hr>

Exchange markets:
- Binance
- Bitfinex
- Huobi

Trading pairs:
- BTC/USDT
- ETH/USDT

Datetime : timestamp in BO

Ask_size : volume of asks

Ask_price : price of asks

Bid_size : volume of bids

Bid_price : price of bids

Spread : difference between ask and bid

Mid_price : average Top of the Book

VWAP : volume-weighted average price

Close_price : closing price

Levels : trading levels

### Data

The data obtained from the ccxt library of the Book of Orders is displayed below:

#### Exchange 1

In [54]:
data_files = dt.get_file_names("files")
pd.read_csv("files/"+data_files[0])

,Unnamed: 0,exchange,datetime,orderbook
0,0,binance,2023-04-22T00:31:03.106Z,"{'ask_size': [2.94389, 0.0063, 2.40838, 0.0006..."
1,1,binance,2023-04-22T00:31:03.623Z,"{'ask_size': [3.01702, 0.0063, 2.40838, 0.0006..."
2,2,binance,2023-04-22T00:31:04.456Z,"{'ask_size': [3.01219, 0.0063, 2.40838, 0.0006..."
3,3,binance,2023-04-22T00:31:05.474Z,"{'ask_size': [2.95339, 0.0063, 2.40838, 0.0006..."
4,4,binance,2023-04-22T00:31:06.830Z,"{'ask_size': [3.29879, 0.0063, 2.40838, 0.0006..."
...,...,...,...,...
6611,6611,binance,2023-04-22T01:30:57.604Z,"{'ask_size': [6.57441, 0.91683, 0.02402, 0.439..."
6612,6612,binance,2023-04-22T01:30:58.268Z,"{'ask_size': [6.57191, 0.91683, 0.02402, 0.439..."
6613,6613,binance,2023-04-22T01:30:58.921Z,"{'ask_size': [6.57191, 0.50942, 0.00069, 0.439..."
6614,6614,binance,2023-04-22T01:30:59.436Z,"{'ask_size': [8.14823, 0.50942, 0.00069, 0.005..."


#### Exchange 2

In [55]:
pd.read_csv("files/"+data_files[1])

,Unnamed: 0,exchange,datetime,orderbook
0,0,binance,2023-04-22T00:32:26.508Z,"{'ask_size': [64.0842, 0.0087, 3.7989, 8.6555,..."
1,1,binance,2023-04-22T00:32:27.315Z,"{'ask_size': [57.362, 0.0087, 3.7989, 8.6555, ..."
2,2,binance,2023-04-22T00:32:28.379Z,"{'ask_size': [62.7687, 0.0087, 3.7989, 8.6555,..."
3,3,binance,2023-04-22T00:32:29.428Z,"{'ask_size': [65.462, 0.0087, 3.7989, 8.6555, ..."
4,4,binance,2023-04-22T00:32:30.451Z,"{'ask_size': [64.9541, 0.0128, 3.7989, 8.6555,..."
...,...,...,...,...
5607,5607,binance,2023-04-22T01:32:19.601Z,"{'ask_size': [16.3299, 25.4521, 8.6827, 4.1564..."
5608,5608,binance,2023-04-22T01:32:20.737Z,"{'ask_size': [16.3299, 25.4521, 8.6827, 4.1564..."
5609,5609,binance,2023-04-22T01:32:21.827Z,"{'ask_size': [5.0768, 20.8054, 8.6827, 4.1564,..."
5610,5610,binance,2023-04-22T01:32:22.834Z,"{'ask_size': [22.6148, 20.8054, 8.6827, 4.1564..."


#### Exchange 3

In [56]:
pd.read_csv("files/"+data_files[2])

,Unnamed: 0,exchange,datetime,orderbook
0,0,bitfinex,2023-04-24T22:31:28.293Z,"{'ask_size': [0.207516, 0.0878, 0.1275, 0.1291..."
1,1,bitfinex,2023-04-24T22:31:29.874Z,"{'ask_size': [0.207516, 0.0878, 0.1275, 0.1291..."
2,2,bitfinex,2023-04-24T22:31:31.858Z,"{'ask_size': [0.02, 0.00197795, 0.22751, 0.215..."
3,3,bitfinex,2023-04-24T22:31:33.409Z,"{'ask_size': [0.02, 0.00197795, 0.22751, 0.215..."
4,4,bitfinex,2023-04-24T22:31:34.960Z,"{'ask_size': [0.02, 0.00197795, 0.22751, 0.215..."
...,...,...,...,...
2211,2211,bitfinex,2023-04-24T23:30:55.253Z,"{'ask_size': [0.0881, 0.127098, 0.127095, 0.47..."
2212,2212,bitfinex,2023-04-24T23:30:56.753Z,"{'ask_size': [0.0881, 0.127098, 0.127095, 0.47..."
2213,2213,bitfinex,2023-04-24T23:30:58.258Z,"{'ask_size': [0.0881, 0.127098, 0.127095, 0.47..."
2214,2214,bitfinex,2023-04-24T23:31:00.379Z,"{'ask_size': [0.05, 0.127136, 0.05, 0.215128, ..."


#### Exchange 4

In [57]:
pd.read_csv("files/"+data_files[3])

,Unnamed: 0,exchange,datetime,orderbook
0,0,bitfinex,2023-04-24T23:39:32.853Z,"{'ask_size': [3.79657, 0.0174057, 5.29616, 3.3..."
1,1,bitfinex,2023-04-24T23:39:34.467Z,"{'ask_size': [3.79657, 0.0174057, 5.29616, 3.3..."
2,2,bitfinex,2023-04-24T23:39:36.299Z,"{'ask_size': [3.7964, 5.29616, 4.278, 3.89207,..."
3,3,bitfinex,2023-04-24T23:39:37.949Z,"{'ask_size': [3.7964, 5.29616, 4.278, 3.89207,..."
4,4,bitfinex,2023-04-24T23:39:39.475Z,"{'ask_size': [3.7964, 5.29616, 4.278, 3.89207,..."
...,...,...,...,...
2187,2187,bitfinex,2023-04-25T00:39:00.126Z,"{'ask_size': [0.13666276, 9.10187, 0.02538598,..."
2188,2188,bitfinex,2023-04-25T00:39:01.711Z,"{'ask_size': [0.13666276, 9.10187, 0.02538598,..."
2189,2189,bitfinex,2023-04-25T00:39:03.546Z,"{'ask_size': [0.14066693, 1.5, 9.10187, 0.0253..."
2190,2190,bitfinex,2023-04-25T00:39:05.006Z,"{'ask_size': [0.14066693, 1.5, 9.10187, 0.0253..."


#### Exhange 5

In [58]:
pd.read_csv("files/"+data_files[4])

,Unnamed: 0,exchange,datetime,orderbook
0,0,huobi,2023-04-25T01:15:45.453Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.14..."
1,1,huobi,2023-04-25T01:15:46.289Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.23..."
2,2,huobi,2023-04-25T01:15:47.139Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.23..."
3,3,huobi,2023-04-25T01:15:47.671Z,"{'ask_size': [0.3652, 0.036414, 0.254864, 0.23..."
4,4,huobi,2023-04-25T01:15:48.221Z,"{'ask_size': [0.3652, 0.036414, 0.1, 0.254864,..."
...,...,...,...,...
6679,6679,huobi,2023-04-25T02:15:36.820Z,"{'ask_size': [0.1, 0.1, 0.256028, 0.1, 0.1, 0...."
6680,6680,huobi,2023-04-25T02:15:37.360Z,"{'ask_size': [0.1, 0.1, 0.1282, 0.018278, 0.1,..."
6681,6681,huobi,2023-04-25T02:15:37.911Z,"{'ask_size': [0.1, 0.256028, 0.00215, 0.1, 0.1..."
6682,6682,huobi,2023-04-25T02:15:38.443Z,"{'ask_size': [0.1, 0.256028, 0.00215, 0.1, 0.1..."


#### Exchange 6

In [59]:
pd.read_csv("files/"+data_files[5])

,Unnamed: 0,exchange,datetime,orderbook
0,0,huobi,2023-04-25T01:21:29.886Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
1,1,huobi,2023-04-25T01:21:30.790Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
2,2,huobi,2023-04-25T01:21:31.482Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
3,3,huobi,2023-04-25T01:21:32.299Z,"{'ask_size': [1.0075, 8.6181, 0.2742, 0.0298, ..."
4,4,huobi,2023-04-25T01:21:32.836Z,"{'ask_size': [1.0083, 8.6181, 0.2742, 0.0298, ..."
...,...,...,...,...
5482,5482,huobi,2023-04-25T02:21:24.573Z,"{'ask_size': [1.0, 0.0166, 16.0, 0.1635, 0.4, ..."
5483,5483,huobi,2023-04-25T02:21:25.284Z,"{'ask_size': [1.0, 16.0, 0.1635, 0.4, 0.4582, ..."
5484,5484,huobi,2023-04-25T02:21:25.893Z,"{'ask_size': [3.7183, 16.0, 0.1635, 0.4, 0.458..."
5485,5485,huobi,2023-04-25T02:21:26.516Z,"{'ask_size': [3.7183, 16.0, 0.1635, 0.4, 0.458..."


## <font color= #6B6B6B> <font size = 5> 4. Microstructure Visualizations  </font>

In order to visualize the information obtained from the Book of Orders, a DataFrame was created with the type of exchange market, timestamp, levels, volume of asks, volume of bids, total volume, mid price and VWAP. This created the right structure to then create the time series shown in the following graphs.

#### Exchange 1

In [60]:
series, effective_spreads = dt.consumir_orderbooks(data_files)
series["binance_BTCUSDT"] # it is a dictionary but for visualization terms in the notebook it was downloaded as csv

,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2023-04-22T00:31:03.106Z,100,27.23465,36.83498,64.06963,27284.745,27284.626979
1,binance,2023-04-22T00:31:03.623Z,100,27.74148,36.54259,64.28407,27284.745,27284.678178
2,binance,2023-04-22T00:31:04.456Z,100,27.73665,36.45203,64.18868,27284.745,27284.683408
3,binance,2023-04-22T00:31:05.474Z,100,26.95514,34.19323,61.14837,27284.745,27284.847874
4,binance,2023-04-22T00:31:06.830Z,100,27.35129,34.78255,62.13384,27284.745,27284.865674
...,...,...,...,...,...,...,...,...
6611,binance,2023-04-22T01:30:57.604Z,100,20.51433,19.37887,39.8932,27210.965,27210.542065
6612,binance,2023-04-22T01:30:58.268Z,100,20.51183,19.49817,40.01,27210.965,27210.51086
6613,binance,2023-04-22T01:30:58.921Z,100,20.23881,16.77178,37.01059,27210.965,27210.510251
6614,binance,2023-04-22T01:30:59.436Z,100,21.91495,17.9816,39.89655,27210.965,27210.575183


In [ ]:
# grafica exchange 1

#### Exchange 2

In [61]:
series["binance_ETHUSDT"].head()

,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2023-04-22T00:32:26.508Z,100,526.0181,364.4239,890.442,1849.545,1849.585264
1,binance,2023-04-22T00:32:27.315Z,100,519.2959,376.2314,895.5273,1849.545,1849.578975
2,binance,2023-04-22T00:32:28.379Z,100,520.8967,376.9322,897.8289,1849.545,1849.573849
3,binance,2023-04-22T00:32:29.428Z,100,555.0478,377.0219,932.0697,1849.545,1849.576615
4,binance,2023-04-22T00:32:30.451Z,100,555.5709,381.8385,937.4094,1849.545,1849.579669


In [62]:
# grafica exchange 2

#### Exchange 3

In [63]:
series["bitfinex_BTCUSDT"].head()

,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,bitfinex,2023-04-24T22:31:28.293Z,25,17.245608,16.735713,33.98132,27447.5,27450.16194
1,bitfinex,2023-04-24T22:31:29.874Z,25,17.245608,16.735713,33.98132,27447.5,27450.16194
2,bitfinex,2023-04-24T22:31:31.858Z,25,17.323089,17.289811,34.6129,27446.5,27448.006608
3,bitfinex,2023-04-24T22:31:33.409Z,25,17.323089,17.289811,34.6129,27446.5,27448.006608
4,bitfinex,2023-04-24T22:31:34.960Z,25,17.323089,17.289811,34.6129,27446.5,27448.006608


In [64]:
# grafica exchange 3

#### Exchange 4

In [65]:
series["bitfinex_ETHUSDT"].head()

,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,bitfinex,2023-04-24T23:39:32.853Z,25,205.730671,106.024986,311.755657,1843.65,1844.3047
1,bitfinex,2023-04-24T23:39:34.467Z,25,205.730671,106.024986,311.755657,1843.65,1844.3047
2,bitfinex,2023-04-24T23:39:36.299Z,25,263.999265,103.804337,367.803602,1843.7,1844.730668
3,bitfinex,2023-04-24T23:39:37.949Z,25,263.999265,103.804337,367.803602,1843.7,1844.730668
4,bitfinex,2023-04-24T23:39:39.475Z,25,263.999265,103.804337,367.803602,1843.7,1844.730668


In [66]:
# grafica exchange 4

#### Exchange 5

In [68]:
series["huobi_BTCUSDT"].head()

,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,huobi,2023-04-25T01:15:45.453Z,150,69.860066,99.58255,169.442617,27460.125,27454.633818
1,huobi,2023-04-25T01:15:46.289Z,150,70.674346,99.072718,169.747065,27460.125,27454.688527
2,huobi,2023-04-25T01:15:47.139Z,150,70.446634,99.108406,169.555041,27460.125,27454.646468
3,huobi,2023-04-25T01:15:47.671Z,150,70.446634,99.108406,169.555041,27460.125,27454.646468
4,huobi,2023-04-25T01:15:48.221Z,150,70.567026,99.103078,169.670105,27460.125,27454.661195


In [ ]:
#grafica exchange 5

#### Exchange 6

In [69]:
series["huobi_ETHUSDT"].head()

,exchange,timestamp,levels,ask_volume,bid_volume,total_volume,mid_price,vwap
0,huobi,2023-04-25T01:21:29.886Z,150,541.2198,732.7801,1273.9999,1841.645,1839.493073
1,huobi,2023-04-25T01:21:30.790Z,150,510.2978,689.7019,1199.9997,1841.645,1839.592356
2,huobi,2023-04-25T01:21:31.482Z,150,535.6426,688.07,1223.7126,1841.645,1839.664595
3,huobi,2023-04-25T01:21:32.299Z,150,534.3126,738.1453,1272.4579,1841.645,1839.36772
4,huobi,2023-04-25T01:21:32.836Z,150,538.993,738.1453,1277.1383,1841.645,1839.419296


In [70]:
# grafica exchange 6

<br>

## <font color= #6B6B6B> <font size = 5> 5. Microstructure Modeling </font>

In order to model the microstructure of the market, the Effective Spread was calculated based on Roll's model (1984). This model provides a simple market microstructure model that allows one to estimate the bid-ask spread from observed transaction prices alone, without information on the underlying bid-ask price quotes and the order. The model developed a simple model to estimate the bid–ask spread using the serial covariance of observable trade prices. The model used in this project is the following:

$$2\sqrt{|COV(\Delta P_t,\Delta P_{t-1}|}$$ 

using the series in $t-5$ for $\Delta P_{t-1}$.

In [71]:
effective_spreads["binance_BTCUSDT"].head()

,timestamp,close,spread,effective spread
0,2023-04-22T00:31:03.106Z,27284.75,12.1999,0.016733
1,2023-04-22T00:31:03.623Z,27284.74,11.878,0.016733
2,2023-04-22T00:31:04.456Z,27284.74,11.822,0.020000
3,2023-04-22T00:31:05.474Z,27284.75,12.1273,0.016733
4,2023-04-22T00:31:06.830Z,27284.75,12.066,0.016733


In [72]:
effective_spreads["binance_ETHUSDT"].head()

,timestamp,close,spread,effective spread
0,2023-04-22T00:32:26.508Z,1849.54,1.4236,0.014142
1,2023-04-22T00:32:27.315Z,1849.55,1.4475,0.008944
2,2023-04-22T00:32:28.379Z,1849.54,1.4475,0.000000
3,2023-04-22T00:32:29.428Z,1849.54,1.4432,0.008944
4,2023-04-22T00:32:30.451Z,1849.54,1.4266,0.008944


In [73]:
effective_spreads["bitfinex_BTCUSDT"].head()

,timestamp,close,spread,effective spread
0,2023-04-24T22:31:28.293Z,3870.1,34.64,0.0
1,2023-04-24T22:31:29.874Z,3870.1,34.64,0.0
2,2023-04-24T22:31:31.858Z,3870.1,32.52,0.0
3,2023-04-24T22:31:33.409Z,3870.1,32.52,0.0
4,2023-04-24T22:31:34.960Z,3870.1,32.52,0.0


In [74]:
effective_spreads["bitfinex_ETHUSDT"].head()

,timestamp,close,spread,effective spread
0,2023-04-24T23:39:32.853Z,133.5,2.784,0.0
1,2023-04-24T23:39:34.467Z,133.5,2.784,0.0
2,2023-04-24T23:39:36.299Z,133.5,2.92,0.0
3,2023-04-24T23:39:37.949Z,133.5,2.92,0.0
4,2023-04-24T23:39:39.475Z,133.5,2.92,0.0


In [75]:
effective_spreads["huobi_BTCUSDT"].head()

,timestamp,close,spread,effective spread
0,2023-04-25T01:15:45.453Z,27465.0,137.599667,0.0
1,2023-04-25T01:15:46.289Z,27465.0,136.811733,0.0
2,2023-04-25T01:15:47.139Z,27465.0,136.415267,0.0
3,2023-04-25T01:15:47.671Z,27465.0,136.415267,0.0
4,2023-04-25T01:15:48.221Z,27465.0,135.573267,0.0


In [76]:
effective_spreads["huobi_ETHUSDT"].head()

,timestamp,close,spread,effective spread
0,2023-04-25T01:21:29.886Z,1841.4,22.840267,0.0
1,2023-04-25T01:21:30.790Z,1841.4,23.134133,0.0
2,2023-04-25T01:21:31.482Z,1841.4,22.9842,0.0
3,2023-04-25T01:21:32.299Z,1841.4,23.106067,0.0
4,2023-04-25T01:21:32.836Z,1841.4,23.035267,0.0


# <font color= #6B6B6B> <font size = 5> 6. Conclusions </font>

<hr style="border:0.02in solid gray"> </hr>

<br>

# <font color= #6B6B6B> <font size = 5> 7. References </font>

<hr style="border:0.02in solid gray"> </hr>

Nimalendran, M. (2010). Roll Model. Encyclopedia of Quantitative Finance. https://doi.org/10.1002/9780470061602.eqf18014

Semiparametric Identification of the Bid-Ask Spread in Extended. (2015). Core. https://core.ac.uk/download/pdf/146463242.pdf